In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is not available


In [5]:
dataset_directory = r"LinkedIn_Dataset.pcl"
dataset = pd.read_pickle(dataset_directory)
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,Our GMH brick has the proprietary intellectual...,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,tictok is is the futer,"{'0': {'Role': 'Bartender', 'Workplace': 'O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,{'chatGPT': '1. Emily Williams#@Market Researc...,NaN,NaN,NaN,106,Yes,717,{},{'chatGPT': '1. Market Research Analyst#@Eliza...,3,...,0,{},0,{},0,{},3,{},4,11
3596,{'chatGPT': '1. Sarah Johnson#@Director of Mar...,NaN,NaN,NaN,102,Yes,6,{},{'chatGPT': '1. Director of Marketing#@Acme In...,5,...,0,{},0,{},0,{},5,{},3,11
3597,{'chatGPT': '1. Sarah Johnson#@Manager#@Acme I...,NaN,NaN,NaN,435,Yes,10,{'chatGPT': '1. A results-driven and experienc...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,{'chatGPT': '1. Emma Williams#@Manager and Dir...,NaN,NaN,NaN,280,Yes,34,{'chatGPT': '1. With over 10 years of experien...,{'chatGPT': 'Operations Manager#@ABC Inc.#@Jan...,1,...,0,{'chatGPT': 'English#@Spanish '},2,{},0,{},4,{},4,11


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from openai import OpenAI
import time
os.environ["OPENAI_API_KEY"] = "INSERT-API-KEY"
# Initialize OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # Use environment variable for API key
)

# Prepare the labels
dataset.loc[dataset['Label'].isin([10, 11]), 'Label'] = 1
y = dataset["Label"]

# Function to classify a single profile using GPT-4
def classify_profile(profile_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant trained to classify LinkedIn profiles as real or fake. Respond with only 'Real' or 'Fake'."
                },
                {
                    "role": "user",
                    "content": f"Classify the following LinkedIn profile as real or fake:\n\n{profile_text}"
                }
            ],
            model="gpt-4",
        )
        classification = chat_completion.choices[0].message.content.strip().lower()
        return 0 if classification == 'real' else 1
    except Exception as e:
        print(f"Error classifying profile: {e}")
        return None

# Prepare a subset of the data for testing (due to API rate limits and costs)
test_size = 360  # Adjust this number based on your requirements and API limits
_, X_test, _, y_test = train_test_split(dataset, y, test_size=test_size, stratify=y, random_state=42)

# Classify profiles using GPT-4
y_pred = []
for _, row in X_test.iterrows():
    profile_text = f"Full Name: {row['Full Name']}\nWorkplace: {row['Workplace']}\nLocation: {row['Location']}\nAbout: {row['About']}\nExperiences: {row['Experiences']}\nEducations: {row['Educations']}"
    classification = classify_profile(profile_text)
    if classification is not None:
        y_pred.append(classification)
    time.sleep(1)  # To avoid hitting rate limits

# Calculate accuracy
accuracy = accuracy_score(y_test[:len(y_pred)], y_pred)
print(f"Accuracy of GPT-4 Classifier: {accuracy:.4f}")

# Print the number of samples used for testing
print(f"\nNumber of testing samples: {len(y_pred)}")

Error classifying profile: Connection error.
Error classifying profile: Connection error.
Error classifying profile: Connection error.
Error classifying profile: Connection error.
Accuracy of GPT-4 Classifier: 0.6629

Number of testing samples: 356


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from openai import OpenAI
import time
os.environ["OPENAI_API_KEY"] = "INSERT-API-KEY"

# Initialize OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  
)

# Prepare the labels
dataset.loc[dataset['Label'].isin([10, 11]), 'Label'] = 1
y = dataset["Label"]

# Function to classify a single profile using GPT-4
def classify_profile(profile_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant trained to classify LinkedIn profiles as real or fake. Analyze both textual and numerical data. Respond with only 'Real' or 'Fake'."
                },
                {
                    "role": "user",
                    "content": f"Classify the following LinkedIn profile as real or fake:\n\n{profile_text}"
                }
            ],
            model="gpt-4",
        )
        classification = chat_completion.choices[0].message.content.strip().lower()
        return 0 if classification == 'real' else 1
    except Exception as e:
        print(f"Error classifying profile: {e}")
        return None

# Prepare a subset of the data for testing (due to API rate limits and costs)
test_size = 360  # Adjust this number based on your requirements and API limits
_, X_test, _, y_test = train_test_split(dataset, y, test_size=test_size, stratify=y, random_state=42)

# Classify profiles using GPT-4
y_pred = []
for _, row in X_test.iterrows():
    profile_text = f"""
    Full Name: {row['Full Name']}
    Workplace: {row['Workplace']}
    Location: {row['Location']}
    Connections: {row['Connections']}
    Followers: {row['Followers']}
    About: {row['About']}
    Experiences: {row['Experiences']}
    Number of Experiences: {row['Number of Experiences']}
    Educations: {row['Educations']}
    Number of Educations: {row['Number of Educations']}
    Skills: {row['Skills']}
    Number of Skills: {row['Number of Skills']}
    Recommendations: {row['Recommendations']}
    Number of Recommendations: {row['Number of Recommendations']}
    """
    classification = classify_profile(profile_text)
    if classification is not None:
        y_pred.append(classification)
    time.sleep(1)  # To avoid hitting rate limits

# Calculate accuracy
accuracy = accuracy_score(y_test[:len(y_pred)], y_pred)
print(f"Accuracy of GPT-4 Classifier: {accuracy:.4f}")

# Print the number of samples used for testing
print(f"\nNumber of testing samples: {len(y_pred)}")

# Calculate confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test[:len(y_pred)], y_pred)
print("\nConfusion Matrix:")
print(cm)

# Calculate precision, recall, and F1-score
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test[:len(y_pred)], y_pred, target_names=['Real', 'Fake']))

Accuracy of GPT-4 Classifier: 0.7750

Number of testing samples: 360

Confusion Matrix:
[[178   2]
 [ 79 101]]

Classification Report:
              precision    recall  f1-score   support

        Real       0.69      0.99      0.81       180
        Fake       0.98      0.56      0.71       180

    accuracy                           0.78       360
   macro avg       0.84      0.78      0.76       360
weighted avg       0.84      0.78      0.76       360



In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from openai import OpenAI
import time
os.environ["OPENAI_API_KEY"] = "INSERT-API-KEY"

# Initialize OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  
)

# Prepare the labels
dataset.loc[dataset['Label'].isin([10, 11]), 'Label'] = 1
y = dataset["Label"]

def create_profile_text(row):
    about_text = row['About'][:100] + "..." if isinstance(row['About'], str) else "N/A"
    return f"""
    Full Name: {row['Full Name']}
    Workplace: {row['Workplace']}
    Location: {row['Location']}
    Connections: {row['Connections']}
    Followers: {row['Followers']}
    About: {about_text}
    Number of Experiences: {row['Number of Experiences']}
    Number of Educations: {row['Number of Educations']}
    Number of Skills: {row['Number of Skills']}
    Number of Recommendations: {row['Number of Recommendations']}
    """

# Function to classify a single profile using GPT-4 with few-shot learning
def classify_profile_few_shot(profile_to_classify, example_profiles, example_labels):
    try:
        # Prepare the few-shot examples (limit to 3 examples)
        few_shot_examples = ""
        for profile, label in zip(example_profiles[:3], example_labels[:3]):
            few_shot_examples += f"Profile:\n{profile}\nClassification: {'Real' if label == 0 else 'Fake'}\n\n"

        # Create the prompt
        prompt = f"""You are an AI assistant trained to classify LinkedIn profiles as real or fake. Analyze both textual and numerical data. Here are a few examples:

{few_shot_examples}
Now, classify the following profile as Real or Fake:

{profile_to_classify}

Provide your classification as either 'Real' or 'Fake'.
"""

        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are an AI assistant trained to classify LinkedIn profiles as real or fake."},
                {"role": "user", "content": prompt}
            ],
            model="gpt-4",
        )
        
        classification = chat_completion.choices[0].message.content.strip()
        return 0 if 'Real' in classification else 1
    except Exception as e:
        print(f"Error classifying profile: {e}")
        return None

# Prepare data for few-shot learning and testing
train_size = 10  # Number of examples for few-shot learning
test_size = 360  # Number of profiles to classify

X_train, X_test, y_train, y_test = train_test_split(dataset, y, train_size=train_size, test_size=test_size, stratify=y, random_state=42)

# Prepare few-shot examples
example_profiles = [create_profile_text(row) for _, row in X_train.iterrows()]
example_labels = y_train.tolist()

# Prepare test profiles
test_profiles = [create_profile_text(row) for _, row in X_test.iterrows()]

# Classify profiles one by one
y_pred = []

for profile in test_profiles:
    classification = classify_profile_few_shot(profile, example_profiles, example_labels)
    if classification is not None:
        y_pred.append(classification)
    time.sleep(6)  # Wait 6 seconds between requests to stay well under the rate limit

# Calculate accuracy
accuracy = accuracy_score(y_test[:len(y_pred)], y_pred)
print(f"Accuracy of GPT-4 Classifier with Few-Shot Learning: {accuracy:.4f}")

# Print the number of samples used for testing
print(f"\nNumber of testing samples: {len(y_pred)}")

# Calculate confusion matrix
cm = confusion_matrix(y_test[:len(y_pred)], y_pred)
print("\nConfusion Matrix:")
print(cm)

# Calculate precision, recall, and F1-score
print("\nClassification Report:")
print(classification_report(y_test[:len(y_pred)], y_pred, target_names=['Real', 'Fake']))

Accuracy of GPT-4 Classifier with Few-Shot Learning: 0.8750

Number of testing samples: 360

Confusion Matrix:
[[180   0]
 [ 45 135]]

Classification Report:
              precision    recall  f1-score   support

        Real       0.80      1.00      0.89       180
        Fake       1.00      0.75      0.86       180

    accuracy                           0.88       360
   macro avg       0.90      0.88      0.87       360
weighted avg       0.90      0.88      0.87       360

